# Examples with `seacarb`'s `errors` routine for uncertainty propagation

<hr>
James Orr - 27 February 2018<br>

<img align="left" width="60%" src="http://www.lsce.ipsl.fr/Css/img/banniere_LSCE_75.png" \><br><br>

LSCE/IPSL, CEA-CNRS-UVSQ, Gif-sur-Yvette, France
<hr>

#### Table of Contents:
    1. Setup, load, install seacarb (one-time only)
    2. Uncertainty propagation with `errors` routine


## 1. Setup 

#### Run interactively

If you are visualizing this after clicking on the link to this file on github, you are visualizing the HTML version of a jupyter notebook. Alternatively, you may run cells interactively and modify them if you have `jupyter notebook` installed on your machine.  To install that software, just download and the anaconda open software installer for your computing platform (Windows, OS X, or Linux) from https://www.continuum.io/downloads and then follow the easy install instructions at

https://docs.continuum.io/anaconda/install#

Then just download this `jupyter notebook` file as well as the seacarb package (see section 1.2 below).  Afterwards, you'll only need to install `Rmagic`, available in the `rpy2` package with the 2 lines of commands shown in the following section.

#### Install `R` and `Rmagic`

Before launching the notebook with the usual command `jupyter notebook`, it is necessary to install the Rmagic functionality.  That is now in the `rpy2` package of conda.  So to install that, at the Unix prompt, just enter the following 2 commands:

`conda install -c r r-essentials`

`conda install rpy2`

That only needs to be done once. Now go to the directory with this notebook, and at the Unix prompt, give the `jupyter notebook` command

#### Documentation for `Rmagic`

Details on using Rmagic are here: https://ipython.org/ipython-doc/2/config/extensions/rmagic.html

### 1.1 Allow use of the `Rmagic` extension (through the `rpy2` package)

In [1]:
%load_ext rpy2.ipython

### 1.2  Install seacarb and call library (need to install `seacarb` only once unlesss updating versions)

#### Uncomment last line in cell below and execute cell if you want to install `seacarb` again

In [2]:
%%R
#install.packages('seacarb')
#install.packages('seacarb', repos="http://ftp.igh.cnrs.fr/pub/CRAN/")

NULL


#### Use seacarb library (only need to invoke this once per session)

In [3]:
%%R
library(seacarb)

/homel/orr/anaconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Loading required package: oce

  warnings.warn(x, RRuntimeWarning)
/homel/orr/anaconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Loading required package: gsw

  warnings.warn(x, RRuntimeWarning)


#### Version number of seacarb that you are using

In [4]:
%%R
packageVersion('seacarb')

[1] ‘3.2.4’


## 2.  Uncertainty Propagation with `errors` routine

In [5]:
%%R
?errors

R Help on ‘errors’errors                 package:seacarb                 R Documentation

_E_r_r_o_r _p_r_o_p_a_g_a_t_i_o_n _f_o_r _c_o_m_p_u_t_e_d _m_a_r_i_n_e _c_a_r_b_o_n_a_t_e _s_y_s_t_e_m _v_a_r_i_a_b_l_e_s

_D_e_s_c_r_i_p_t_i_o_n:

     Estimates uncertainties in computed carbonate system variables by
     propagating standard error (uncertainty) in six input variables,
     including:

        • the input pair of carbonate system variables,

        • the 2 input nutrients (silicate and phosphate
          concentrations),

        • temperature and salinity. It also accounts for

        • the errors in the key dissociation constants pK0, pK1, pK2,
          pKb, pKw, pKspa and pKspc

        • the error in total boron

_U_s_a_g_e:

     errors(flag, var1, var2, S=35, T=25, Patm=1, P=0, Pt=0, Sit=0, 
                   evar1=0, evar2=0, eS=0.01, eT=0.01, ePt=0, eSit=0, 
                   epK=c(0.004, 0.01

### 2.1 `carb` routine computes derived variables but does not to propagate uncertainties

In [6]:
%%R
# Standard seacarb routine to compute derived variables (carb, not errors)
vars <- carb(flag=15, var1=2300e-6, var2=2000e-6, S=35, T=18, P=0, Patm=1.0, Pt=0, Sit=0, 
            pHscale="T", kf="pf", k1k2="l", ks="d", b="u74")

print(vars)

  flag  S  T Patm P       pH          CO2     fCO2     pCO2  fCO2pot  pCO2pot
1   15 35 18    1 0 8.152525 1.018509e-05 297.0486 298.0854 297.0854 298.1224
  fCO2insitu pCO2insitu        HCO3          CO3   DIC    ALK OmegaAragonite
1   297.0486   298.0854 0.001779217 0.0002105975 0.002 0.0023       3.252296
  OmegaCalcite
1     5.029397


### 2.2 Use the `errors` routine to propagate uncertainties 

It is used just like the `carb` routine except that a middle line is added to specify the input uncertainties

#### 2.2.1 $A_\text{T}$-$C_\text{T}$ pair with default uncertainties for the constants (epK) and total boron (eBt)

By default:
* epK=c(0.004, 0.015, 0.03, 0.01, 0.01, 0.02, 0.02)
* eBt=0.01

It is strongly recommended to use the default values for epK and eBt. These non-zero values lead to significant propagated uncertainties in the calculated variables. It is NOT justified to consider that those uncertainties are negligible, as assumed is some previous work. However, in later examples we do make calculations with epK=NULL and eBt=NULL in order to show the relative importance of the default unertainties for those parameters.

#### The easiest way to use the default uncertainties for epK and eBt is simply not to include those arguments in the call to errors (as below).

In [7]:
%%R
err <- errors(flag=15, var1=2300e-6, var2=2000e-6, S=35, T=25, P=0, Patm=1.0, Pt=0, Sit=0, 
            evar1=c(0e-6,5e-6), evar2=c(0e-6,5e-6), eS=0, eT=0, ePt=0, eSit=0, 
            pHscale="T", kf="pf", k1k2="l", ks="d", b="u74")

#print(err)
err

             H         pH          CO2     fCO2     pCO2         HCO3
1 4.271496e-10 0.02062246 6.243867e-07 22.29153 22.36292 4.844416e-06
2 4.885838e-10 0.02358846 7.145240e-07 25.42889 25.51033 9.832047e-06
           CO3 OmegaAragonite OmegaCalcite
1 4.354922e-06      0.1706290    0.2588683
2 6.391204e-06      0.1860801    0.2823097


##### Do exactly the same but specify the default values explicitly

In [8]:
%%R
epK=c(0.004, 0.015, 0.03, 0.01, 0.01, 0.02, 0.02)
eBt=0.01
err <- errors(flag=15, var1=2300e-6, var2=2000e-6, S=35, T=25, P=0, Patm=1.0, Pt=0, Sit=0, 
            evar1=c(0,5e-6), evar2=c(0,5e-6), eS=0, eT=0, ePt=0, eSit=0, epK=epK, eBt=eBt,
            pHscale="T", kf="pf", k1k2="l", ks="d", b="u74")

#print(err)
err

             H         pH          CO2     fCO2     pCO2         HCO3
1 4.271496e-10 0.02062246 6.243867e-07 22.29153 22.36292 4.844416e-06
2 4.885838e-10 0.02358846 7.145240e-07 25.42889 25.51033 9.832047e-06
           CO3 OmegaAragonite OmegaCalcite
1 4.354922e-06      0.1706290    0.2588683
2 6.391204e-06      0.1860801    0.2823097


#### 2.2.3 $A_\text{T}$-$C_\text{T}$ pair but with zero uncertainties for the constants and total boron (unrealistic)

In [9]:
%%R
epK=NULL
eBt=NULL
err <- errors(flag=15, var1=2300e-6, var2=2000e-6, S=35, T=25, P=0, Patm=1.0, Pt=0, Sit=0, 
            evar1=c(0e-6,5e-6), evar2=c(0e-6,5e-6), eS=0, eT=0, ePt=0, eSit=0, epK=epK, eBt=eBt,
            pHscale="T", kf="pf", k1k2="l", ks="d", b="u74")

err

             H         pH          CO2     fCO2     pCO2         HCO3
1 0.000000e+00 0.00000000 0.000000e+00  0.00000  0.00000 0.000000e+00
2 2.371863e-10 0.01145118 3.473987e-07 12.23585 12.27503 8.555746e-06
           CO3 OmegaAragonite OmegaCalcite
1 0.000000e+00     0.00000000     0.000000
2 4.677835e-06     0.07423961     0.112632


#####  Another way to specify the NULL values for epK and eBt

In [10]:
%%R
epK=rep(0.0, 7)
eBt=0.0
err <- errors(flag=15, var1=2300e-6, var2=2000e-6, S=35, T=25, P=0, Patm=1.0, Pt=0, Sit=0, 
            evar1=c(0,5e-6), evar2=c(0,5e-6), eS=0, eT=0, ePt=0, eSit=0, epK=epK, eBt=eBt,
            pHscale="T", kf="pf", k1k2="l", ks="d", b="u74")

#print(err)
err

             H         pH          CO2     fCO2     pCO2         HCO3
1 0.000000e+00 0.00000000 0.000000e+00  0.00000  0.00000 0.000000e+00
2 2.371863e-10 0.01145118 3.473987e-07 12.23585 12.27503 8.555746e-06
           CO3 OmegaAragonite OmegaCalcite
1 0.000000e+00     0.00000000     0.000000
2 4.677835e-06     0.07423961     0.112632


##### Specify the NULL values for epK but default value for eBt

In [11]:
%%R
epK=rep(0.0, 7)
eBt=0.01
err <- errors(flag=15, var1=2300e-6, var2=2000e-6, S=35, T=25, P=0, Patm=1.0, Pt=0, Sit=0, 
            evar1=c(0,5e-6), evar2=c(0,5e-6), eS=0, eT=0, ePt=0, eSit=0, epK=epK, eBt=eBt,
            pHscale="T", kf="pf", k1k2="l", ks="d", b="l10")

#print(err)
err

             H          pH          CO2      fCO2      pCO2         HCO3
1 3.024785e-11 0.001460343 4.158664e-08  1.464737  1.469428 6.035883e-07
2 2.416396e-10 0.011517751 3.536942e-07 12.457582 12.497480 8.519193e-06
           CO3 OmegaAragonite OmegaCalcite
1 6.451749e-07     0.01023925   0.01553439
2 4.664307e-06     0.07402491   0.11230623


#### 2.2.4 Propagate uncertainties with a different input pair (pCO$_2$-pH) 

In [12]:
%%R
epK=c(0.004, 0.015, 0.03, 0.01, 0.01, 0.02, 0.02)
eBt = 0.01

err <- errors(flag=21, var1=400, var2=8.05, S=35, T=18, P=0, Patm=1.0, Pt=0, Sit=0, 
            evar1=2, evar2=0.01, eS=0, eT=0, ePt=0, eSit=0, epK=epK, eBt=eBt, method='mo',r=0.0,
            pHscale="T", kf="pf", k1k2="l", ks="d", b="u74")
err

           CO2     fCO2         HCO3          CO3          DIC          ALK
1 1.432296e-07 1.992793 8.071361e-05 1.595326e-05 9.135438e-05 0.0001042291
  OmegaAragonite OmegaCalcite
1      0.2764148    0.4274517


#### 2.2.5 For more complex examples of using seacarb's `errors` routine

Just cut & paste what you like from the output of the cell below into a new jupyter notebook cell & try them out, interactively

In [13]:
%%R
example(errors)


errors> ## 1) For the input pair ALK and DIC (var1 and var2 when flag=15),
errors> ## compute resulting uncertainty from given uncertainty on ALK and DIC (5 umol/kg)
errors> ## and default uncertainties in dissociation constants and total boron
errors> ## using the default method (Gaussian)
errors> errors(flag=15, var1=2300e-6, var2=2000e-6, S=35, T=25, P=0, Patm=1.0, Pt=0, Sit=0, 
errors+        evar1=5e-6, evar2=5e-6, eS=0, eT=0, ePt=0, eSit=0, 
errors+        pHscale="T", kf="pf", k1k2="l", ks="d", b="u74")
             H         pH         CO2     fCO2     pCO2         HCO3
1 4.885838e-10 0.02358846 7.14524e-07 25.42889 25.51033 9.832047e-06
           CO3 OmegaAragonite OmegaCalcite
1 6.391204e-06      0.1860801    0.2823097

errors> ## Typical output:
errors> ## H             pH          CO2           fCO2      pCO2      HCO3          ...
errors> ## 3.721614e-10  0.01796767  5.441869e-07  19.25338  19.31504  9.170116e-06  ...
errors> 
errors> ## 2) Do the same as in one, but ass